This is a script for converting the variables in the EUPPBench dataset from netCDF format into a dataframe of records by flattening. This data preparation is for the validation (2017) and test (2018) sets. Please refer to the Chapter 3 of the report for more information on data splitting and processings of input features.

In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import datetime

In [1]:
# Change paths for own use
ROOT = "ROOT/"
data_path = ROOT+"raw_data_EUPPBench/"
export_path = ROOT+"preprocessed_data/"

In [3]:
# Loading the data (input and output features) in netCDF format
vis_forecast_obs = xr.open_dataarray(data_path + "vis_forecast_observations.nc")
vis_forecast = xr.open_dataarray(data_path + "vis_forecasts.nc")
t_forecast = xr.open_dataarray(data_path + "t_forecasts.nc")
q_forecast = xr.open_dataarray(data_path + "q_forecasts.nc")
tp6_forecast = xr.open_dataarray(data_path + "tp6_forecasts.nc")
tcc_forecast = xr.open_dataarray(data_path + "tcc_forecasts.nc")
u10_forecast = xr.open_dataarray(data_path + "u10_forecasts.nc")
v10_forecast = xr.open_dataarray(data_path + "v10_forecasts.nc")
dis_from_coast = xr.open_dataarray(data_path + "distance_to_coastline.nc")

/Users/clifflau/opt/anaconda3/lib/python3.8/site-packages/gribapi/__init__.py:23: UserWarning: ecCodes 2.31.0 or higher is recommended. You are running version 2.30.0
  warnings.warn(


In [4]:
vis_forecast['forecast_reference_time']

/Users/clifflau/opt/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


<xarray.DataArray 'forecast_reference_time' (forecast_reference_time: 730)>
array(['2017-01-01T00:00:00.000000000', '2017-01-02T00:00:00.000000000',
       '2017-01-03T00:00:00.000000000', ..., '2018-12-29T00:00:00.000000000',
       '2018-12-30T00:00:00.000000000', '2018-12-31T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * forecast_reference_time  (forecast_reference_time) datetime64[ns] 2017-01...
    day_of_year              (forecast_reference_time) int64 ...
    surface                  float64 ...
Attributes:
    axis:           T
    standard_name:  forecast_reference_time

In [5]:
# Data structure
# vis_obs['forecast_period', spot_index', 'forecast_reference_time']
# vis_reforecast['forecast_period', 'realization', 'spot_index', 'forecast_reference_time']
vis_forecast_obs

<xarray.DataArray 'visibility_in_air' (forecast_period: 21, spot_index: 118,
                                       forecast_reference_time: 730)>
[1808940 values with dtype=float32]
Coordinates:
  * forecast_period          (forecast_period) timedelta64[ns] 0 days 00:00:0...
  * spot_index               (spot_index) int64 0 1 2 3 4 ... 114 115 116 117
  * forecast_reference_time  (forecast_reference_time) datetime64[ns] 2017-01...
    altitude                 (spot_index) float64 ...
    day_of_year              (forecast_reference_time) int64 ...
    land_usage               (spot_index) int8 ...
    latitude                 (spot_index) float64 ...
    longitude                (spot_index) float64 ...
    wmo_id                   (spot_index) int64 ...
Attributes:
    standard_name:  visibility_in_air
    units:          m

In [5]:
# Create arrays for latitude, longitudes, base times, lead times and orecasted/observed values
lat = np.array(vis_forecast_obs['latitude'])
lon = np.array(vis_forecast_obs['longitude'])
alt = np.array(vis_forecast_obs['altitude'])
dis_from_coast = np.array(dis_from_coast)[0:118] # consider first 119 stations only
base= np.array(vis_forecast_obs['forecast_reference_time'])
lead = np.array(vis_forecast_obs['forecast_period'][1:])
lead_hr = np.array(lead*10**-9/3600, dtype='int')
vis_forecast_obs_array = np.array(vis_forecast_obs[1:, 0:118, :]) # consider lead time from 6 to 12 hrs
vis_forecast_array = np.array(vis_forecast[1:, :, 0:118, :]) 
t_forecast_array = np.array(t_forecast[1:, :, 0:118, :])
q_forecast_array = np.array(q_forecast[1:, :, 0:118, :])*1000 # g/kg
tp6_forecast_array = np.array(tp6_forecast[:, :, 0:118, :])*1000 # mm
tcc_forecast_array = np.array(tcc_forecast[1:, :, 0:118, :])
u10_forecast_array = np.array(u10_forecast[1:, :, 0:118, :])
v10_forecast_array = np.array(v10_forecast[1:, :, 0:118, :])
wind_forecast_array = np.sqrt(u10_forecast_array**2+v10_forecast_array**2)

In [6]:
# Compute proportions of members predicting each visibility index (input feature)
prop_VP = []
prop_P = []
prop_M = []
prop_G = []
prop_VG = []
prop_E = []
for l in range(len(lead)):
    for s in range(len(lon)):
        for b in range(len(base)):
            prop_VP.append(sum(vis_forecast_array[l,:,s,b]<1000)/51)
            prop_P.append(sum((vis_forecast_array[l,:,s,b]>=1000)&(vis_forecast_array[l,:,s,b]<4000))/51)
            prop_M.append(sum((vis_forecast_array[l,:,s,b]>=4000)&(vis_forecast_array[l,:,s,b]<10000))/51)
            prop_G.append(sum((vis_forecast_array[l,:,s,b]>=10000)&(vis_forecast_array[l,:,s,b]<20000))/51)
            prop_VG.append(sum((vis_forecast_array[l,:,s,b]>=20000)&(vis_forecast_array[l,:,s,b]<40000))/51)
            prop_E.append(sum(vis_forecast_array[l,:,s,b]>=40000)/51)

In [7]:
# Compute ensemble means and standard deviations of input features
import time

st = time.time()

vis_forecast_ensemble_mean = np.copy(vis_forecast_array)
vis_forecast_ensemble_std = np.copy(vis_forecast_array)
t_forecast_ensemble_mean = np.copy(t_forecast_array)
t_forecast_ensemble_std = np.copy(t_forecast_array)
q_forecast_ensemble_mean = np.copy(q_forecast_array)
q_forecast_ensemble_std = np.copy(q_forecast_array)
tp6_forecast_ensemble_mean = np.copy(tp6_forecast_array)
tp6_forecast_ensemble_std = np.copy(tp6_forecast_array)
tcc_forecast_ensemble_mean = np.copy(tcc_forecast_array)
tcc_forecast_ensemble_std = np.copy(tcc_forecast_array)
u10_forecast_ensemble_mean = np.copy(u10_forecast_array)
u10_forecast_ensemble_std = np.copy(u10_forecast_array)
v10_forecast_ensemble_mean = np.copy(v10_forecast_array)
v10_forecast_ensemble_std = np.copy(v10_forecast_array)
wind_forecast_ensemble_mean = np.copy(wind_forecast_array)
wind_forecast_ensemble_std = np.copy(wind_forecast_array)

for l in range(len(lead)):
    for s in range(len(lon)):
        for b in range(len(base)):
            vis_forecast_ensemble_mean[l,:,s,b] = np.mean(vis_forecast_ensemble_mean[l,:,s,b])
            vis_forecast_ensemble_std[l,:,s,b] = np.std(vis_forecast_ensemble_std[l,:,s,b])
            t_forecast_ensemble_mean[l,:,s,b] = np.mean(t_forecast_ensemble_mean[l,:,s,b])
            t_forecast_ensemble_std[l,:,s,b] = np.std(t_forecast_ensemble_std[l,:,s,b])
            q_forecast_ensemble_mean[l,:,s,b] = np.mean(q_forecast_ensemble_mean[l,:,s,b])
            q_forecast_ensemble_std[l,:,s,b] = np.std(q_forecast_ensemble_std[l,:,s,b])
            tp6_forecast_ensemble_mean[l,:,s,b] = np.mean(tp6_forecast_ensemble_mean[l,:,s,b])
            tp6_forecast_ensemble_std[l,:,s,b] = np.std(tp6_forecast_ensemble_std[l,:,s,b])
            tcc_forecast_ensemble_mean[l,:,s,b] = np.mean(tcc_forecast_ensemble_mean[l,:,s,b])
            tcc_forecast_ensemble_std[l,:,s,b] = np.std(tcc_forecast_ensemble_std[l,:,s,b])
            u10_forecast_ensemble_mean[l,:,s,b] = np.mean(u10_forecast_ensemble_mean[l,:,s,b])
            u10_forecast_ensemble_std[l,:,s,b] = np.std(u10_forecast_ensemble_std[l,:,s,b])
            v10_forecast_ensemble_mean[l,:,s,b] = np.mean(v10_forecast_ensemble_mean[l,:,s,b])
            v10_forecast_ensemble_std[l,:,s,b] = np.std(v10_forecast_ensemble_std[l,:,s,b])
            wind_forecast_ensemble_mean[l,:,s,b] = np.mean(wind_forecast_ensemble_mean[l,:,s,b])
            wind_forecast_ensemble_std[l,:,s,b] = np.std(wind_forecast_ensemble_std[l,:,s,b])
            
vis_forecast_ensemble_mean = vis_forecast_ensemble_mean[:, 0, :, :]
vis_forecast_ensemble_std = vis_forecast_ensemble_std[:, 0, :, :]
t_forecast_ensemble_mean = t_forecast_ensemble_mean[:, 0, :, :]
t_forecast_ensemble_std = t_forecast_ensemble_std[:, 0, :, :]
q_forecast_ensemble_mean = q_forecast_ensemble_mean[:, 0, :, :]
q_forecast_ensemble_std = q_forecast_ensemble_std[:, 0, :, :]
tp6_forecast_ensemble_mean = tp6_forecast_ensemble_mean[:, 0, :, :]
tp6_forecast_ensemble_std = tp6_forecast_ensemble_std[:, 0, :, :]
tcc_forecast_ensemble_mean = tcc_forecast_ensemble_mean[:, 0, :, :]
tcc_forecast_ensemble_std = tcc_forecast_ensemble_std[:, 0, :, :]
u10_forecast_ensemble_mean = u10_forecast_ensemble_mean[:, 0, :, :]
u10_forecast_ensemble_std = u10_forecast_ensemble_std[:, 0, :, :]
v10_forecast_ensemble_mean = v10_forecast_ensemble_mean[:, 0, :, :]
v10_forecast_ensemble_std = v10_forecast_ensemble_std[:, 0, :, :]
wind_forecast_ensemble_mean = wind_forecast_ensemble_mean[:, 0, :, :]
wind_forecast_ensemble_std = wind_forecast_ensemble_std[:, 0, :, :]

et = time.time()
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

/Users/clifflau/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:239: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)


Execution time: 508.5145161151886 seconds


In [8]:
# Create arrays for writting a dataframe of records
base_df = np.tile(base, len(lat)*len(lead_hr))
lon_df = np.tile(np.repeat(lon, len(base)), len(lead_hr))
lat_df = np.tile(np.repeat(lat, len(base)), len(lead_hr))
alt_df = np.tile(np.repeat(alt, len(base)), len(lead_hr))
dis_from_coast_df = np.tile(np.repeat(dis_from_coast, len(base)), len(lead_hr))
lead_df = np.repeat(lead, len(base)*len(lat))
lead_hr_df = np.repeat(lead_hr, len(base)*len(lat))

forecast_time_df = base_df + lead_df
forecast_time_pd = pd.DatetimeIndex(forecast_time_df)

# Forecast time (time of day)
time_in_day = [i.hour for i in forecast_time_pd]
for i in range(len(time_in_day)):
    if time_in_day[i]==0 or time_in_day[i]==6:
        time_in_day[i] = '0' + str(time_in_day[i])
    else:
        time_in_day[i] = str(time_in_day[i])

# Forecast time (day of year)
day_in_year = np.array([(i-np.datetime64(str(i.year)+"-01-01")).days+1 
                        for i in forecast_time_pd])
day_in_year_sin = np.sin(((day_in_year-1)/366)*2*np.pi)
day_in_year_cos = np.cos(((day_in_year-1)/366)*2*np.pi)
        
vis_obs_df = vis_forecast_obs_array.flatten()
vis_ensemble_mean_df = vis_forecast_ensemble_mean.flatten()
vis_ensemble_std_df = vis_forecast_ensemble_std.flatten()
t_ensemble_mean_df = t_forecast_ensemble_mean.flatten()
t_ensemble_std_df = t_forecast_ensemble_std.flatten()
q_ensemble_mean_df = q_forecast_ensemble_mean.flatten()
q_ensemble_std_df = q_forecast_ensemble_std.flatten()
tp6_ensemble_mean_df = tp6_forecast_ensemble_mean.flatten()
tp6_ensemble_std_df = tp6_forecast_ensemble_std.flatten()
tcc_ensemble_mean_df = tcc_forecast_ensemble_mean.flatten()
tcc_ensemble_std_df = tcc_forecast_ensemble_std.flatten()
u10_ensemble_mean_df = u10_forecast_ensemble_mean.flatten()
u10_ensemble_std_df = u10_forecast_ensemble_std.flatten()
v10_ensemble_mean_df = v10_forecast_ensemble_mean.flatten()
v10_ensemble_std_df = v10_forecast_ensemble_std.flatten()
wind_ensemble_mean_df = wind_forecast_ensemble_mean.flatten()
wind_ensemble_std_df = wind_forecast_ensemble_std.flatten()

In [9]:
# Create the dataframe
df = pd.DataFrame({'base': base_df,
                   'lead': lead_df,
                   'lead_hr': lead_hr_df,
                   'forecast_time': forecast_time_df,
                   'time_in_day': time_in_day,
                   'day_in_year': day_in_year,
                   'day_in_year_sin': day_in_year_sin,
                   'day_in_year_cos': day_in_year_cos,
                   'station_lat': lat_df,
                   'station_lon': lon_df,
                   'station_alt': alt_df,
                   'dis_from_coast': dis_from_coast_df,
                   't_ensemble_mean': t_ensemble_mean_df,
                   't_ensemble_std': t_ensemble_std_df,
                   'q_ensemble_mean': q_ensemble_mean_df,
                   'q_ensemble_std': q_ensemble_std_df,
                   'tp6_ensemble_mean': tp6_ensemble_mean_df,
                   'tp6_ensemble_std': tp6_ensemble_std_df,
                   'tcc_ensemble_mean': tcc_ensemble_mean_df,
                   'tcc_ensemble_std': tcc_ensemble_std_df,
                   'u10_ensemble_mean': u10_ensemble_mean_df,
                   'u10_ensemble_std': u10_ensemble_std_df,
                   'v10_ensemble_mean': v10_ensemble_mean_df,
                   'v10_ensemble_std': v10_ensemble_std_df,
                   'wind_ensemble_mean': wind_ensemble_mean_df,
                   'wind_ensemble_std': wind_ensemble_std_df,
                   'vis_ensemble_mean': vis_ensemble_mean_df,
                   'vis_ensemble_std': vis_ensemble_std_df,
                   'member_prop_VP': prop_VP,
                   'member_prop_P': prop_P,
                   'member_prop_M': prop_M,
                   'member_prop_G': prop_G,
                   'member_prop_VG': prop_VG,
                   'member_prop_E': prop_E,
                   'vis_obs': vis_obs_df
                  })
df

,base,lead,lead_hr,forecast_time,time_in_day,day_in_year,day_in_year_sin,day_in_year_cos,station_lat,station_lon,...,wind_ensemble_std,vis_ensemble_mean,vis_ensemble_std,member_prop_VP,member_prop_P,member_prop_M,member_prop_G,member_prop_VG,member_prop_E,vis_obs
0,2017-01-01,0 days 06:00:00,6,2017-01-01 06:00:00,06,1,0.000000,1.000000,52.928000,4.781000,...,0.392337,18645.101562,2341.318115,0.000000,0.000000,0.000000,0.725490,0.274510,0.000000,1700.0
1,2017-01-02,0 days 06:00:00,6,2017-01-02 06:00:00,06,2,0.017166,0.999853,52.928000,4.781000,...,0.518166,23898.185547,5.029546,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,15000.0
2,2017-01-03,0 days 06:00:00,6,2017-01-03 06:00:00,06,3,0.034328,0.999411,52.928000,4.781000,...,0.601479,19213.099609,2691.179443,0.000000,0.000000,0.000000,0.568627,0.431373,0.000000,13000.0
3,2017-01-04,0 days 06:00:00,6,2017-01-04 06:00:00,06,4,0.051479,0.998674,52.928000,4.781000,...,0.589314,21881.076172,1392.999756,0.000000,0.000000,0.000000,0.098039,0.901961,0.000000,13000.0
4,2017-01-05,0 days 06:00:00,6,2017-01-05 06:00:00,06,5,0.068615,0.997643,52.928000,4.781000,...,1.100902,22317.501953,1376.300903,0.000000,0.000000,0.000000,0.039216,0.960784,0.000000,20000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1722795,2018-12-27,5 days 00:00:00,120,2019-01-01 00:00:00,00,1,0.000000,1.000000,45.786833,3.149333,...,0.657104,17238.462891,17450.683594,0.235294,0.137255,0.098039,0.196078,0.098039,0.235294,6715.0
1722796,2018-12-28,5 days 00:00:00,120,2019-01-02 00:00:00,00,2,0.017166,0.999853,45.786833,3.149333,...,0.488838,16497.964844,18977.689453,0.274510,0.215686,0.058824,0.058824,0.196078,0.196078,15318.0
1722797,2018-12-29,5 days 00:00:00,120,2019-01-03 00:00:00,00,3,0.034328,0.999411,45.786833,3.149333,...,0.603335,44745.261719,6543.390625,0.000000,0.000000,0.000000,0.000000,0.176471,0.823529,33728.0
1722798,2018-12-30,5 days 00:00:00,120,2019-01-04 00:00:00,00,4,0.051479,0.998674,45.786833,3.149333,...,1.081511,35814.320312,12468.176758,0.019608,0.000000,0.019608,0.078431,0.431373,0.450980,38492.0


In [10]:
# Check for extreme values
max(df['t_ensemble_mean']), max(df['q_ensemble_mean']), max(df['tp6_ensemble_mean']), max(df['wind_ensemble_mean']), max(df['vis_ensemble_mean']), max(df['vis_obs'])

(298.8223571777344,
 8.17647933959961,
 35.20281219482422,
 21.365615844726562,
 1.954747134183279e+35,
 9.969209968386869e+36)

In [11]:
# Dropping extreme observations
df_drop = df.drop(df[(df.vis_obs>100000) | (df.vis_ensemble_mean>100000)].index)

In [12]:
df_drop.shape

(1298207, 35)

In [13]:
# Validation period: 2017
# Testing period: 2018
df_valid = df_drop[df_drop['base']<=np.datetime64('2017-12-31')]
df_test = df_drop[df_drop['base']>np.datetime64('2017-12-31')]

In [14]:
df_valid

,base,lead,lead_hr,forecast_time,time_in_day,day_in_year,day_in_year_sin,day_in_year_cos,station_lat,station_lon,...,wind_ensemble_std,vis_ensemble_mean,vis_ensemble_std,member_prop_VP,member_prop_P,member_prop_M,member_prop_G,member_prop_VG,member_prop_E,vis_obs
0,2017-01-01,0 days 06:00:00,6,2017-01-01 06:00:00,06,1,0.000000,1.000000,52.928000,4.781000,...,0.392337,18645.101562,2341.318115,0.000000,0.000000,0.000000,0.725490,0.274510,0.000000,1700.0
1,2017-01-02,0 days 06:00:00,6,2017-01-02 06:00:00,06,2,0.017166,0.999853,52.928000,4.781000,...,0.518166,23898.185547,5.029546,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,15000.0
2,2017-01-03,0 days 06:00:00,6,2017-01-03 06:00:00,06,3,0.034328,0.999411,52.928000,4.781000,...,0.601479,19213.099609,2691.179443,0.000000,0.000000,0.000000,0.568627,0.431373,0.000000,13000.0
3,2017-01-04,0 days 06:00:00,6,2017-01-04 06:00:00,06,4,0.051479,0.998674,52.928000,4.781000,...,0.589314,21881.076172,1392.999756,0.000000,0.000000,0.000000,0.098039,0.901961,0.000000,13000.0
4,2017-01-05,0 days 06:00:00,6,2017-01-05 06:00:00,06,5,0.068615,0.997643,52.928000,4.781000,...,1.100902,22317.501953,1376.300903,0.000000,0.000000,0.000000,0.039216,0.960784,0.000000,20000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1722430,2017-12-27,5 days 00:00:00,120,2018-01-01 00:00:00,00,1,0.000000,1.000000,45.786833,3.149333,...,1.510165,39495.140625,17696.080078,0.000000,0.019608,0.098039,0.098039,0.156863,0.627451,60000.0
1722431,2017-12-28,5 days 00:00:00,120,2018-01-02 00:00:00,00,2,0.017166,0.999853,45.786833,3.149333,...,1.964946,23193.568359,19647.101562,0.039216,0.117647,0.294118,0.078431,0.176471,0.294118,59895.0
1722432,2017-12-29,5 days 00:00:00,120,2018-01-03 00:00:00,00,3,0.034328,0.999411,45.786833,3.149333,...,1.532669,24519.666016,15525.642578,0.000000,0.039216,0.235294,0.215686,0.254902,0.254902,60000.0
1722433,2017-12-30,5 days 00:00:00,120,2018-01-04 00:00:00,00,4,0.051479,0.998674,45.786833,3.149333,...,1.563531,33135.472656,18358.226562,0.000000,0.078431,0.117647,0.098039,0.254902,0.450980,59260.0


In [15]:
df_test

,base,lead,lead_hr,forecast_time,time_in_day,day_in_year,day_in_year_sin,day_in_year_cos,station_lat,station_lon,...,wind_ensemble_std,vis_ensemble_mean,vis_ensemble_std,member_prop_VP,member_prop_P,member_prop_M,member_prop_G,member_prop_VG,member_prop_E,vis_obs
365,2018-01-01,0 days 06:00:00,6,2018-01-01 06:00:00,06,1,0.000000,1.000000,52.928000,4.781000,...,0.345179,48256.843750,690.082581,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,19000.0
366,2018-01-02,0 days 06:00:00,6,2018-01-02 06:00:00,06,2,0.017166,0.999853,52.928000,4.781000,...,0.455442,39858.246094,5088.321289,0.000000,0.000000,0.000000,0.000000,0.470588,0.529412,18000.0
367,2018-01-03,0 days 06:00:00,6,2018-01-03 06:00:00,06,3,0.034328,0.999411,52.928000,4.781000,...,0.979747,24173.501953,9506.372070,0.000000,0.000000,0.039216,0.372549,0.470588,0.117647,13000.0
368,2018-01-04,0 days 06:00:00,6,2018-01-04 06:00:00,06,4,0.051479,0.998674,52.928000,4.781000,...,0.337588,43666.417969,2167.463379,0.000000,0.000000,0.000000,0.000000,0.039216,0.960784,9000.0
369,2018-01-05,0 days 06:00:00,6,2018-01-05 06:00:00,06,5,0.068615,0.997643,52.928000,4.781000,...,0.606247,46836.132812,1929.764526,0.000000,0.000000,0.000000,0.000000,0.019608,0.980392,25000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1722795,2018-12-27,5 days 00:00:00,120,2019-01-01 00:00:00,00,1,0.000000,1.000000,45.786833,3.149333,...,0.657104,17238.462891,17450.683594,0.235294,0.137255,0.098039,0.196078,0.098039,0.235294,6715.0
1722796,2018-12-28,5 days 00:00:00,120,2019-01-02 00:00:00,00,2,0.017166,0.999853,45.786833,3.149333,...,0.488838,16497.964844,18977.689453,0.274510,0.215686,0.058824,0.058824,0.196078,0.196078,15318.0
1722797,2018-12-29,5 days 00:00:00,120,2019-01-03 00:00:00,00,3,0.034328,0.999411,45.786833,3.149333,...,0.603335,44745.261719,6543.390625,0.000000,0.000000,0.000000,0.000000,0.176471,0.823529,33728.0
1722798,2018-12-30,5 days 00:00:00,120,2019-01-04 00:00:00,00,4,0.051479,0.998674,45.786833,3.149333,...,1.081511,35814.320312,12468.176758,0.019608,0.000000,0.019608,0.078431,0.431373,0.450980,38492.0


In [16]:
# Save into a csv files
df_valid.to_csv(export_path + "df_valid.csv", index=False)
df_test.to_csv(export_path + "df_test.csv", index=False)

In [17]:
# Check the records in the dataframe are matched correctly
i = 127182
print(df_test.iloc[i])
l = np.asarray(lead==df_test.iloc[i]['lead']).nonzero()[0]
s = np.asarray(lat==df_test.iloc[i]['station_lat']).nonzero()[0]
b = np.asarray(base==df_test.iloc[i]['base']).nonzero()[0]

base                  2018-10-22 00:00:00
lead                      1 days 00:00:00
lead_hr                                24
forecast_time         2018-10-23 00:00:00
time_in_day                            00
day_in_year                           296
day_in_year_sin                  -0.93871
day_in_year_cos                  0.344707
station_lat                     49.209667
station_lon                      4.155333
station_alt                          95.0
dis_from_coast                 209443.625
t_ensemble_mean                280.269012
t_ensemble_std                   0.569195
q_ensemble_mean                  0.499198
q_ensemble_std                   0.089212
tp6_ensemble_mean                0.000019
tp6_ensemble_std                 0.001026
tcc_ensemble_mean                0.113248
tcc_ensemble_std                 0.169114
u10_ensemble_mean               -0.010946
u10_ensemble_std                 0.268654
v10_ensemble_mean               -2.458661
v10_ensemble_std                 0

In [18]:
{'base': base[b],
 'lead': int(lead[l]*10**-9/3600),
 'lead_hr': int(lead_hr[l]),
 #'forecast_time': forecast_time_df,
 #'time_in_day': time_in_day,
 #'day_in_year_sin': day_in_year_sin,
 #'day_in_year_cos': day_in_year_cos,
 'station_lat': float(lat[s]),
 'station_lon': float(lon[s]),
 'station_alt': float(alt[s]),
 'dis_from_coast': float(dis_from_coast[s]),
 't_ensemble_mean': float(np.mean(t_forecast_array[l,:,s,b])),
 't_ensemble_std': float(np.std(t_forecast_array[l,:,s,b])),
 'q_ensemble_mean': float(np.mean(q_forecast_array[l,:,s,b])),
 'q_ensemble_std': float(np.std(q_forecast_array[l,:,s,b])),
 'tp6_ensemble_mean': float(np.mean(tp6_forecast_array[l,:,s,b])),
 'tp6_ensemble_std': float(np.std(tp6_forecast_array[l,:,s,b])),
 'tcc_ensemble_mean': float(np.mean(tcc_forecast_array[l,:,s,b])),
 'tcc_ensemble_std': float(np.std(tcc_forecast_array[l,:,s,b])),
 'u10_ensemble_mean': float(np.mean(u10_forecast_array[l,:,s,b])),
 'u10_ensemble_std': float(np.std(u10_forecast_array[l,:,s,b])),
 'v10_ensemble_mean': float(np.mean(v10_forecast_array[l,:,s,b])),
 'v10_ensemble_std': float(np.std(v10_forecast_array[l,:,s,b])),
 'wind_ensemble_mean': float(np.mean(wind_forecast_array[l,:,s,b])),
 'wind_ensemble_std': float(np.std(wind_forecast_array[l,:,s,b])),
 'vis_obs': float(vis_forecast_obs_array[l,s,b]),
 'vis_ensemble_mean': float(np.mean(vis_forecast_array[l,:,s,b])),
 'vis_ensemble_std': float(np.std(vis_forecast_array[l,:,s,b]))
}

{'base': array(['2018-10-22T00:00:00.000000000'], dtype='datetime64[ns]'),
 'lead': 24,
 'lead_hr': 24,
 'station_lat': 49.209667,
 'station_lon': 4.155333,
 'station_alt': 95.0,
 'dis_from_coast': 209443.625,
 't_ensemble_mean': 280.2690124511719,
 't_ensemble_std': 0.5691947340965271,
 'q_ensemble_mean': 0.499197781085968,
 'q_ensemble_std': 0.08921197801828384,
 'tp6_ensemble_mean': 1.869949664978776e-05,
 'tp6_ensemble_std': 0.0010255783563479781,
 'tcc_ensemble_mean': 0.11324773728847504,
 'tcc_ensemble_std': 0.1691139191389084,
 'u10_ensemble_mean': -0.010946123860776424,
 'u10_ensemble_std': 0.2686540484428406,
 'v10_ensemble_mean': -2.4586613178253174,
 'v10_ensemble_std': 0.3380683362483978,
 'wind_ensemble_mean': 2.4728615283966064,
 'wind_ensemble_std': 0.3413994610309601,
 'vis_obs': 19437.0,
 'vis_ensemble_mean': 44117.9375,
 'vis_ensemble_std': 2513.135009765625}